This notebook ingests NCA geotifs for the 'public website', applies colours to them based on examples provided alongside the data, and then generates static map tilesets, using the xyz tile schema. These tilesets are then uploaded to a dedicated S3 bucket.
Resources:
link to matplotlib colormaps - https://matplotlib.org/stable/gallery/color/colormap_reference.html

In [16]:
import os
import numpy as np
import math
import rasterio
from rasterio.plot import reshape_as_image
from matplotlib.colors import Normalize
from matplotlib.cm import get_cmap
import gdal2tiles
import folium
from IPython.display import display

def calculate_min_zoom_level(dataset):
    # Earth's radius in meters
    earth_radius = 6378137

    # Calculate pixel size in meters at the equator
    pixel_size_x = (dataset.bounds.right - dataset.bounds.left) / dataset.width
    pixel_size_y = (dataset.bounds.top - dataset.bounds.bottom) / dataset.height
    pixel_size = max(pixel_size_x, pixel_size_y)

    # Calculate ground resolution at the equator
    equator_circumference = 2 * math.pi * earth_radius
    ground_res_at_equator = pixel_size / equator_circumference * 360

    # Find the closest zoom level where the ground resolution is higher than the tile resolution
    min_zoom_level = 0
    for zoom_level in range(21):  # Web maps typically use zoom levels 0 to 20
        tile_ground_res = 360 / (256 * 2**zoom_level)
        if tile_ground_res <= ground_res_at_equator:
            min_zoom_level = zoom_level
            break

    return min_zoom_level

def generate_coloured_tif(data, colour_map_name, original_nodata):
    """
    Apply a color map to the input data and handle nodata values.
    """
    cmap = get_cmap(colour_map_name)
    norm_data = (data - np.nanmin(data)) / (np.nanmax(data) - np.nanmin(data))
    norm_data = np.nan_to_num(norm_data, nan=original_nodata)
    colored_data = cmap(norm_data)[:, :, :3]  # Only take the RGB channels
    return (colored_data * 255).astype(np.uint8)

def process_geotiff(input_geotiff_path, output_directory, colour_map_name):
    try:
        with rasterio.open(input_geotiff_path) as src:
            tif_data = src.read(1, masked=True).astype('float32')
            original_nodata = src.nodata

            # Generate the colored GeoTIFF image
            coloured_image = generate_coloured_tif(tif_data, colour_map_name, original_nodata)

            # Define metadata for the colored GeoTIFF
            meta = src.meta.copy()
            meta.update(count=3, dtype='uint8', nodata=0)  # 3 channels, uint8, no nodata

            # Save the colored GeoTIFF locally
            file_name = os.path.splitext(os.path.basename(input_geotiff_path))[0]
            coloured_geotiff_path = os.path.join(output_directory, f"{file_name}-coloured.tif")
            with rasterio.open(coloured_geotiff_path, 'w', **meta) as dst:
                dst.write(coloured_image.transpose(2, 0, 1))  # Reorder to match band order in rasterio

            return {
                'message': 'Colored GeoTIFF created successfully.',
                'coloured_geotiff_path': coloured_geotiff_path
            }
    except Exception as e:
        return {
            'error': str(e)
        }

def create_map_tiles(geotiff_path, output_directory):
    with rasterio.open(geotiff_path) as dataset:
        min_zoom = calculate_min_zoom_level(dataset)
        print(f"Min zoom level calculated: {min_zoom}")

    tiles_output_directory = os.path.join(output_directory, 'map_tiles')
    os.makedirs(tiles_output_directory, exist_ok=True)

    try:
        gdal2tiles.generate_tiles(
            geotiff_path,
            tiles_output_directory,
            zoom=[15, 16],  # Adjust max zoom as needed
            tmscompatible=True,
            #srcnodata=0,
            #nodata=0,
            verbose=False
        )
        print("Tiles generated successfully.")
    except Exception as e:
        print("An error occurred during tile generation:", e)

    return tiles_output_directory, min_zoom



# Usage example:
input_geotiff_path = 'data/Orana/Orana Rasters/groundcover/YQ-2022-GC10.tif'
output_directory = 'data/Orana/output_rasters/'
colour_map_name = 'RdYlGn'

processing_result = process_geotiff(input_geotiff_path, output_directory, colour_map_name)

if 'coloured_geotiff_path' in processing_result:
    tiles_directory, min_zoom = create_map_tiles(processing_result['coloured_geotiff_path'], output_directory)
    print("Tiles directory:", tiles_directory)
else:
    print("Error in processing GeoTIFF:", processing_result.get('error', 'Unknown Error'))



Min zoom level calculated: 14
Generating Base Tiles:
0

/tmp/ipykernel_12586/298237734.py:39: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap(colour_map_name)
/opt/conda/lib/python3.11/site-packages/matplotlib/colors.py:732: RuntimeWarning: overflow encountered in multiply
  xa *= self.N


...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
Tiles generated successfully.
Tiles directory: data/Orana/output_rasters/map_tiles


The below cell has all the NCA tifs listed in order, and the colormaps for eah one listed in matching order afterwards. The transparency issue for the map tiles should be the only thing that needs to be bug fixed.Note - change the minimum zoom for the maptiles to be set dynamically based on function that calcualtes min zoom. Set max zoom at 18 or so - higher levels will take a long time to generate.

In [17]:
input_tifs = [
    'birds/YQ_Allbirds_gam.tif',
    'birds/YQ_Grassbirds_gam.tif',
    'birds/YQ_Woodbirds_gam.tif',
    'groundcover/YQ-2022-GC10.tif',
    'groundcover/YQ-2022-MEANGC10.tif',
    'invertebrates/YQdet_map.tif',
    'plants/YQ_AllNativePlants_gam.tif',
    'plants/YQ_AllPlants_gam.tif',
    'plants/YQ_NativeGroundLayer_gam.tif',
    'plants/YQ_NativeShrubs_gam.tif',
    'trees-shade-shelter/YQ-NCI5-Proximity.tif',
    'trees-shade-shelter/YQ-shade-footprint-onoffsitecontribution-production-only.tif',
    'trees-shade-shelter/YQ-summer-shelter-onoffcontrib-production-zone.tif',
    'trees-shade-shelter/YQ-winter-shelter-onoffcontrib-production-zone.tif'
]

#the cmaps are matplotlib in same order as the above geotifs to make sure the right colours are applied.
cmap_for_input_rasters=[
    'Blues',    ``
    'Blues',
    'Blues',
    'RdYlGn',
    'RdYlGn',
    'YlOrBr',
    'Purples',
    'Purples',
    'Purples',
    'Purples',
    'Greens', #these are only 2 shades, aka discrete map with 2 bins
    'Reds', #these are only 2 shades
    'Blues' #these are only 2 shades


]